In [9]:
import numpy as np
import pandas as pd
from collections import Counter
from pandas import DataFrame
import numpy as np
from bs4 import BeautifulSoup
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
 
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
download('stopwords')
download('wordnet')
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer
import tweepy 
from tweepy import OAuthHandler 
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to C:\Users\Smita Roy
[nltk_data]     Trivedi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Smita Roy
[nltk_data]     Trivedi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Remember to keep your Keys and Tokens a secret!
consumer_key= ' consumer_secret= ' access_token= ' access_token_secret= 'V en_secret = 'xxxxx'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)

In [7]:
user = "..."
tweets = api.user_timeline(id=user, count=1000)
json_data = [t._json for t in tweets]

df = pd.json_normalize(json_data)
df.head(2)

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.place.id,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.contained_within,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates
0,Tue Dec 21 10:59:37 +0000 2021,1473246776874262529,1473246776874262529,India continues to be stock pickers' paradise!...,True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Dec 19 05:57:16 +0000 2021,1472445911666069507,1472445911666069507,"Advance Tax at Rs.4,60,000 Cr, up 53%\nNet Dir...",False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
import csv


labels1 = {}

with open(r'C:\Users\Smita Roy Trivedi\Desktop\bull.csv', mode='r') as inp:
    reader = csv.reader(inp)
    labels1 = {rows[0]:rows[1] for rows in reader}

print(labels1)

{'buy': 'MB', 'bought': 'MB', 'bull': 'MB', 'bullish': 'MB', 'long': 'MB', 'support': 'MB', 'buying': 'MB', 'bulls': 'MB', 'sell': 'MS', 'selling': 'MS', 'sold': 'MS', 'resistance': 'MS', 'short': 'MS', 'bear': 'MS', 'bearish': 'MS', 'advance': 'MB', 'decline': 'MS', 'up': 'MB', 'down': 'MS', 'not buy': 'MS', 'not buying': 'MS', 'not bought': 'MS', 'not sell': 'MB', 'not selling': 'MB', 'not sold': 'MB', 'hold': 'MH', 'wait': 'MH', 'sideways': 'MH'}


In [12]:
def matcher1(k):
    x = (i for i in labels1 if i in k)
    return ' | '.join(map(labels1.get, x))

In [13]:
def preprocess(text):
    sentence = text.lower()
    sentence=sentence.replace('{html}',"")
    sentence=sentence.replace('{https}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 ] 
    return filtered_words

In [14]:
df['filtered_words'] = df['text'].apply(preprocess)
df.head(2)

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.contained_within,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates,filtered_words
0,Tue Dec 21 10:59:37 +0000 2021,1473246776874262529,1473246776874262529,India continues to be stock pickers' paradise!...,True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[india, continues, stock, pickers, paradise, o..."
1,Sun Dec 19 05:57:16 +0000 2021,1472445911666069507,1472445911666069507,"Advance Tax at Rs.4,60,000 Cr, up 53%\nNet Dir...",False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[advance, tax, net, direct, tax]"


In [16]:
df['filtered_words'] = df['filtered_words'].astype(str,'ignore')
df['flag_list1']=df['filtered_words'].apply(matcher1)
df['flag_list1'] = df['flag_list1'].astype(str,'ignore')

In [18]:
df['number1']=df['flag_list1'].str.count(r'[a-z]')
df['negation']=df['filtered_words'].str.findall(r'\'\b(not|dont|neither|nor)\b\'')
df['negation'] = df['negation'].astype(str,'ignore')
df['count_negation']=df['negation'].str.count(r'[a-z]')
df['count_flag_pos']=df['flag_list1'].str.findall(r'(MB)')
df['count_flag_pos'] = df['count_flag_pos'].astype(str,'ignore')
df['count_flag_pos1']=df['count_flag_pos'].str.count(r'[A-Z]')
df['count_flag_neg']=df['flag_list1'].str.findall(r'(MS)')
df['count_flag_neg'] = df['count_flag_neg'].astype(str,'ignore')
df['count_flag_neg1']=df['count_flag_neg'].str.count(r'[A-Z]')
df['count_flag_hold']=df['flag_list1'].str.findall(r'(MH)')
df['count_flag_hold'] = df['count_flag_hold'].astype(str,'ignore')
df['count_flag_hold1']=df['count_flag_hold'].str.count(r'[A-Z]')
df['sentiment']=(df['count_flag_pos1']+1)/(df['count_flag_neg1']+1)
df.head(2)

,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,number1,negation,count_negation,count_flag_pos,count_flag_pos1,count_flag_neg,count_flag_neg1,count_flag_hold,count_flag_hold1,sentiment
0,Tue Dec 21 10:59:37 +0000 2021,1473246776874262529,1473246776874262529,India continues to be stock pickers' paradise!...,True,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,0,[],0,[],0,"['MS', 'MS']",4,[],0,0.2
1,Sun Dec 19 05:57:16 +0000 2021,1472445911666069507,1472445911666069507,"Advance Tax at Rs.4,60,000 Cr, up 53%\nNet Dir...",False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,...,0,[],0,['MB'],2,[],0,[],0,3.0
